In [4]:
import pandas as pd
import numpy as np
import shap

In [4]:
for dataset in ["compas", "cc", "german"]:
    df = pd.read_csv(rf'C:\Users\shrey\Desktop\DSC 261\DSC-261-FINAL\Data\{dataset}_RBF_train.csv')
    df = pd.get_dummies(df)
    num_rows, num_cols = df.shape
    # Assuming self.generator_specs["perturbation_std"] is defined, replace it with the actual value if needed
    perturbation_std = 0.3  # Example: 0.1 or any other value

    # Generate new data for the entire DataFrame
    df_generated = np.random.normal(0, perturbation_std, size=(num_rows, num_cols))

    # Convert the generated data into a DataFrame, ensuring it has the same column names as the original DataFrame
    df_new = pd.DataFrame(df_generated, columns=df.columns)

    df_new.drop(columns=['response'], inplace=True)
    output_file_path = rf'C:\Users\shrey\Desktop\DSC 261\DSC-261-FINAL\Data\{dataset}_adversarial_train_perturbation.csv'  # Specify your output file path
    df_new.to_csv(output_file_path, index=False)

In [3]:
from copy import deepcopy
dataset = "compas"
# Assuming X is your dataset
# If X is a DataFrame, convert it to a NumPy array
df = pd.read_csv(rf'C:\Users\shrey\Desktop\DSC 261\DSC-261-FINAL\Data\{dataset}_RBF_train.csv')
df = pd.get_dummies(df)
X = df.to_numpy()

# Perform SHAP k-means clustering to obtain the background distribution
n_kmeans = 32  # Adjust based on your dataset size and diversity
kmeans_summary = shap.kmeans(X, n_kmeans)
background_distribution = kmeans_summary.data
n_samples = len(X)  # Number of new samples to generate
new_instances = []  # List to store new instances

for _ in range(int(n_samples)):
    i = np.random.choice(X.shape[0])
    point = deepcopy(X[i, :])

    # Iterate over points, sampling and updating
    for _ in range(X.shape[1]):
        j = np.random.choice(X.shape[1])
        point[j] = deepcopy(background_distribution[np.random.choice(background_distribution.shape[0]), j])

    new_instances.append(point)

# Convert the list of new instances into a numpy array
substituted_training_data = np.vstack(new_instances)


# Convert the numpy array back to a pandas DataFrame if needed
# Assuming you know the column names, or they are simply feature indices
df_substituted = pd.DataFrame(substituted_training_data, columns=df.columns)
# Desired order for the columns
new_order = ['age', 'two_year_recid', 'priors_count', 'length_of_stay', 
             'c_charge_degree_F', 'c_charge_degree_M', 'sex_Female', 'sex_Male', 
             'race', 'unrelated_column_one', 'unrelated_column_two']

# Reindex the DataFrame using the new order
df_substituted = df[new_order]
output_file_path = rf'C:\Users\shrey\Desktop\DSC 261\DSC-261-FINAL\Data\shap_{dataset}_adversarial_train_perturbation.csv'  # Specify your output file path
df_substituted.to_csv(output_file_path, index=False)

In [7]:
from copy import deepcopy
dataset = "german"
# Assuming X is your dataset
# If X is a DataFrame, convert it to a NumPy array
df = pd.read_csv(rf'C:\Users\shrey\Desktop\DSC 261\DSC-261-FINAL\Data\shap_{dataset}_RBF_train.csv')
df = pd.get_dummies(df)
X = df.to_numpy()

# Perform SHAP k-means clustering to obtain the background distribution
n_kmeans = 12  # Adjust based on your dataset size and diversity
kmeans_summary = shap.kmeans(X, n_kmeans)
background_distribution = kmeans_summary.data
n_samples = len(X)  # Number of new samples to generate
new_instances = []  # List to store new instances

for _ in range(int(n_samples)):
    i = np.random.choice(X.shape[0])
    point = deepcopy(X[i, :])

    # Iterate over points, sampling and updating
    for _ in range(X.shape[1]):
        j = np.random.choice(X.shape[1])
        point[j] = deepcopy(background_distribution[np.random.choice(background_distribution.shape[0]), j])

    new_instances.append(point)

# Convert the list of new instances into a numpy array
substituted_training_data = np.vstack(new_instances)


# Convert the numpy array back to a pandas DataFrame if needed
# Assuming you know the column names, or they are simply feature indices
df_substituted = pd.DataFrame(substituted_training_data, columns=df.columns)
# Desired order for the columns
# new_order = ['age', 'two_year_recid', 'priors_count', 'length_of_stay', 
#              'c_charge_degree_F', 'c_charge_degree_M', 'sex_Female', 'sex_Male', 
#              'race', 'unrelated_column_one', 'unrelated_column_two']

# Reindex the DataFrame using the new order
# df_substituted = df[new_order]
output_file_path = rf'C:\Users\shrey\Desktop\DSC 261\DSC-261-FINAL\Data\shap_{dataset}_adversarial_train_perturbation.csv'  # Specify your output file path
df_substituted.to_csv(output_file_path, index=False)

In [8]:
df_substituted

,Gender,ForeignWorker,Single,Age,LoanDuration,LoanAmount,LoanRateAsPercentOfIncome,YearsAtCurrentHome,NumberOfOtherLoansAtBank,NumberOfLiableIndividuals,...,response,CheckingAccountBalance_geq_0_lt_200,CheckingAccountBalance_geq_200,CheckingAccountBalance_lt_0,SavingsAccountBalance_geq_100_lt_500,SavingsAccountBalance_geq_500,SavingsAccountBalance_lt_100,YearsAtCurrentJob_geq_1_lt_4,YearsAtCurrentJob_geq_4,YearsAtCurrentJob_lt_1
0,1,0,1,37,36,4844,4,2,1,1,...,1,0,0,1,0,0,1,1,0,0
1,1,0,1,33,14,1213,4,3,2,1,...,1,0,0,1,0,1,1,0,0,0
2,1,0,0,42,18,3213,1,3,1,1,...,1,1,0,0,0,1,0,0,0,0
3,1,0,0,33,36,2859,3,4,1,1,...,0,0,0,1,0,0,1,0,0,0
4,1,0,1,38,39,2360,1,2,1,1,...,1,1,0,1,0,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
895,1,0,1,27,24,14896,4,3,2,1,...,1,0,0,1,0,0,0,0,1,0
896,1,0,0,34,18,1245,4,2,1,1,...,1,1,0,0,0,0,1,0,0,0
897,1,1,0,54,36,2859,3,3,1,1,...,1,0,0,1,0,0,1,1,0,0
898,0,0,1,32,39,4844,3,4,1,1,...,1,0,0,1,0,0,1,0,0,0
